# Classic Warcraftlogs Raid Log Wrapper
Extract raid log data to csv format using the classic.warcraftlogs.com API

As of July 2020, only general raid log information is included in the API, no detailed events

## Default values

In [1]:
event_types = ['summary','damage-done','damage-taken', 'healing', 'casts', 
                'summons', 'buffs', 'debuffs', 'deaths', 'threat', 'resources', 
                'interrupts', 'dispels']
guild = 'nerds of prey'
server = 'mankrik'
auth_url = 'https://classic.warcraftlogs.com/oauth/authorize'
token_url = 'https://classic.warcraftlogs.com/oauth/token'
config_path = '../wow_config.ini'

## Package Imports

In [2]:
import os
import matplotlib.pyplot as pyplot
import seaborn as sns
import pandas as pd 
import numpy as np 
import requests
import configparser as cp
import json
from datetime import datetime as dt
from IPython.display import HTML

## Function definitions
Can also be found in the module wowlogs_lib.py

#### Get credentials

In [3]:
def get_credentials (auth_url, token_url, config_path):
    """Get client, key, and secret from ini file"""
    config = cp.ConfigParser()
    config.read(config_path)
    client_id = config.get('client', 'wowlogs')
    key = config.get('key', 'wowlogs')
    secret = config.get('secret', 'wowlogs')
    try:
        r = requests.post(token_url, 
            data={'grant_type': 'client_credentials'}, 
            auth=(client_id, secret))
        result = json.loads(r.text)
        return result, key, secret, client_id
    except:
        print('Connection Error')
        return 'Connection Error', key, secret, client_id
    
token, key, secret, client_id = get_credentials (auth_url, token_url, config_path)  

#### Format names for urls

In [4]:
def format_name (name):
    """ Format any name for Wowlogs' API"""
    for i, s in enumerate(name):
        if i == 0:
            name = name.replace(name[i], name[i].upper())
        elif name[i] == ' ':
            try:
                if name[i+1:i+3] != 'of':
                    name = name.replace(name[i+1], name[i+1].upper())
            except:
                continue
    name = name.replace(' ', "%20")
    return name

#### Get a list of raid logs (reports) for a specific guild

In [5]:
def get_guild_reports (key, guild, server):
    """Fetch a list of raid logs"""
    url = 'https://classic.warcraftlogs.com:443/v1/reports/guild/' \
            + guild + '/' + server + '/US?api_key=' + key
    tmp = pd.DataFrame()
    try:
        r  = requests.get (url)
        result = json.loads(r.text)
        for r in result: # Saves the results to a dataframe
            r['end'] = dt.utcfromtimestamp(r['end']/1000).strftime('%Y-%m-%d')
            r['start'] = dt.utcfromtimestamp(r['start']/1000).strftime('%Y-%m-%d')
            tmp = tmp.append(r, ignore_index = True)
        return tmp
    except:
            print('Connection Error')
            return tmp

#### Get overall fight data for a raid log (report)

In [6]:
def get_report_fights (report):
    """Get the general data from each fight in a raid log"""
    url = 'https://classic.warcraftlogs.com:443/v1/report/fights/' + \
            report + '?api_key=' + key
    fights = pd.DataFrame()
    players = pd.DataFrame()
    enemies = pd.DataFrame()
    r = requests.get(url)
    result = json.loads(r.text)
    for fight in result['fights']:
        fight['time_to_kill'] = dt.utcfromtimestamp((fight['end_time'] - fight['start_time'])/1000).strftime('%HH:%MM:%SS')
        fight['start_time'] = dt.utcfromtimestamp(fight['start_time']/1000).strftime('%H:%M:%S')
        fight['end_time'] = dt.utcfromtimestamp(fight['end_time']/1000).strftime('%H:%M:%S')
        fights = fights.append(fight, ignore_index = True)
    for friend in result['friendlies']:
        players = players.append(friend, ignore_index = True)
    for enemy in result['enemies']:
        enemies = enemies.append(enemy, ignore_index = True) 
    players = players.drop('fights', axis = 1)
    enemies = enemies.drop('fights', axis = 1)
    fights.to_csv(os.path.join(working_dir,'data', report + '_fights.csv'), index = False)
    players.to_csv(os.path.join(working_dir,'data',report + '_players.csv'), index = False)
    enemies.to_csv(os.path.join(working_dir,'data',report + '_enemies.csv'), index = False)
    return fights, players.dropna(), enemies

#### Get detailed reports for specific event types (damage done, healing, etc)
Currently these APIs are empty or contain mostly NaN 

In [7]:
def get_report_events(report, event):
    """Get detailed reports on group performance """
    tmp = pd.DataFrame()
    url = 'https://classic.warcraftlogs.com:443/v1/report/events/' + \
            event + '/' + report + '?api_key=' + key
    try:
        r = requests.get(url)
        result = json.loads(r.text)
        for res in result['events']:
            tmp = tmp.append(res, ignore_index = True)
        tmp.to_csv(os.path.join(working_dir,'data', report + '_' + event + '_.csv'), index = False)
        display(HTML('<hr><h4>' + event + '</h4>'))
        display(HTML(tmp.to_html()))
    except:
        print("Connection Error: Event = ", event)
    return tmp

## Default Variables

In [8]:
working_dir = os.getcwd()
guild = format_name (guild)
server = format_name (server)

## Get a list of raid logs

In [9]:
reports = get_guild_reports (key, guild, server)
display(HTML(reports.to_html()))

,end,id,owner,start,title,zone
0,2020-07-27,DAYcb3KyCgLpxFh1,Kazenokami,2020-07-27,ZG 7/26,1003.0
1,2020-07-26,vYkFCb3GrA4MTa79,kaveli,2020-07-26,Blackwing Lair,1002.0
2,2020-07-26,AXc79Kz3DfwJnGpb,Kazenokami,2020-07-26,BWL 7/25,1002.0
3,2020-07-23,vMXw2NA3HjkqLdB1,Kazenokami,2020-07-23,MC 7/22,1000.0
4,2020-07-23,HC1QK7bL9M42mx8c,kaveli,2020-07-23,Molten Core,1000.0
5,2020-07-19,3Xm29HBtafY8Pz4w,kaveli,2020-07-19,Blackwing Lair,1002.0
6,2020-07-19,C4qnXh1PDkAN9c6H,Kazenokami,2020-07-19,BWL 7/18,1002.0
7,2020-07-17,48FZzAjBc3mdvNX7,Kazenokami,2020-07-17,ZG 7/16,1003.0
8,2020-07-16,qT1aQPHMwnmD3fFp,Kazenokami,2020-07-16,MC 7/15,1000.0
9,2020-07-16,mRw1zgxHdBW6bVK2,kaveli,2020-07-16,Molten Core,1000.0


## Get fight info

In [10]:
fights, players, enemies = get_report_fights('VwrG1Y84ZPkNf69X')
display(HTML('<hr><h4>Fights</h4>'))
display(HTML(fights.to_html()))
display(HTML('<hr><h4>Players</h4>'))
display(HTML(players.to_html()))
display(HTML('<hr><h4>Enemies</h4>'))
display(HTML(enemies.to_html()))

,boss,end_time,id,name,originalBoss,start_time,time_to_kill,bossPercentage,difficulty,fightPercentage,hasWorldBuffs,kill,lastPhaseForPercentageDisplay,maps,partial,size
0,0.0,00:06:01,1.0,Razorgore the Untamed,610.0,00:00:00,00H:06M:01S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,610.0,00:06:45,2.0,Razorgore the Untamed,NaN,00:06:01,00H:00M:44S,0.0,3.0,0.0,1.0,1.0,0.0,[],2.0,40.0
2,611.0,00:13:55,3.0,Vaelastrasz the Corrupt,NaN,00:12:49,00H:01M:05S,0.0,3.0,0.0,1.0,1.0,0.0,[],3.0,40.0
3,612.0,00:37:08,4.0,Broodlord Lashlayer,NaN,00:35:17,00H:01M:50S,0.0,3.0,0.0,1.0,1.0,0.0,[],3.0,40.0
4,613.0,00:50:21,5.0,Firemaw,NaN,00:47:23,00H:02M:58S,0.0,3.0,0.0,1.0,1.0,0.0,[],3.0,40.0
5,614.0,01:15:26,6.0,Ebonroc,NaN,01:13:25,00H:02M:00S,0.0,3.0,0.0,1.0,1.0,0.0,[],3.0,40.0
6,615.0,01:18:49,7.0,Flamegor,NaN,01:17:21,00H:01M:27S,0.0,3.0,0.0,1.0,1.0,0.0,[],3.0,40.0
7,616.0,01:31:50,8.0,Chromaggus,NaN,01:25:28,00H:06M:21S,0.0,3.0,0.0,1.0,1.0,0.0,[],3.0,40.0
8,0.0,01:40:37,9.0,Nefarian,617.0,01:38:16,00H:02M:20S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,617.0,01:45:46,10.0,Nefarian,NaN,01:40:37,00H:05M:08S,0.0,3.0,0.0,0.0,1.0,0.0,[],2.0,40.0


,guid,icon,id,name,server,type
0,33817470.0,Hunter-Marksmanship,153.0,Elontusker,Mankrik,Hunter
2,12792946.0,Warrior,39.0,Brigade,Mankrik,Warrior
3,9857240.0,Hunter-Marksmanship,3.0,Konz,Mankrik,Hunter
4,29439767.0,Mage-Frost,7.0,Mizzu,Mankrik,Mage
5,25479221.0,Warlock-Destruction,44.0,Fritzina,Mankrik,Warlock
6,10945591.0,Hunter-Marksmanship,13.0,Kaeleb,Mankrik,Hunter
7,4159.0,Druid,23.0,Draco,Mankrik,Druid
9,25398462.0,Shaman-Restoration,161.0,Smu,Mankrik,Shaman
10,29439770.0,Hunter-Marksmanship,40.0,Imanzi,Mankrik,Hunter
11,459937.0,Druid-Restoration,48.0,Akela,Mankrik,Druid


,guid,icon,id,name,type
0,14302.0,custom-icon-spell_fire_fire,122.0,Chromatic Drakonid,NPC
1,14302.0,custom-icon-spell_fire_fire,119.0,Chromatic Drakonid,NPC
2,12017.0,custom-icon-spell_holy_excorcism_02,96.0,Broodlord Lashlayer,Boss
3,14263.0,NPC,123.0,Bronze Drakonid,NPC
4,14263.0,custom-icon-trade_engineering,124.0,Bronze Drakonid,NPC
5,11983.0,custom-icon-spell_fire_fireball,101.0,Firemaw,Boss
6,14605.0,custom-icon-ability_backstab,152.0,Bone Construct,NPC
7,12457.0,custom-icon-spell_shadow_siphonmana,66.0,Blackwing Spellbinder,NPC
8,14264.0,NPC,117.0,Red Drakonid,NPC
9,13020.0,custom-icon-spell_fire_sealoffire,91.0,Vaelastrasz the Corrupt,Boss


## Get detailed fight stats
API is empty as of July 2020 with a note saying it's still under development

In [11]:
event_list = []
for event in event_types:
    event_list.append(get_report_events(reports.id[0], event))

,difficulty,encounterID,name,size,timestamp,type,agility,armor,artifact,auras,avoidance,block,critMelee,critRanged,critSpell,dodge,faction,gear,hasteMelee,hasteRanged,hasteSpell,heartOfAzeroth,intellect,leech,mastery,parry,pvpTalents,sourceID,specID,speed,stamina,strength,talents,versatilityDamageDone,versatilityDamageReduction,versatilityHealingDone,ability,source,sourceIsFriendly,targetID,targetIsFriendly,amount,attackPower,classResources,facing,hitPoints,hitType,itemLevel,mapID,maxHitPoints,mitigated,resourceActor,spellPower,unmitigatedAmount,x,y
0,3.0,785.0,High Priestess Jeklik,20.0,0.0,encounterstart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,0.0,combatantinfo,346.0,1902.0,[],"[{'source': 0, 'ability': 24425, 'stacks': 1, 'icon': 'ability_creature_poison_05.jpg', 'name': 'Spirit of Zandalar'}, {'source': 0, 'ability': 22888, 'stacks': 1, 'icon': 'inv_misc_head_dragon_01.jpg', 'name': 'Rallying Cry of the Dragonslayer'}]",0.0,0.0,0.0,0.0,0.0,0.0,98.0,"[{'id': 16900, 'quality': 4, 'icon': 'inv_helmet_09.jpg', 'itemLevel': 76}, {'id': 19885, 'quality': 4, 'icon': 'inv_jewelry_ring_45.jpg', 'itemLevel': 66}, {'id': 18810, 'quality': 4, 'icon': 'inv_shoulder_18.jpg', 'itemLevel': 71}, {'id': 0, 'quality': 1, 'icon': 'inv_axe_02.jpg', 'itemLevel': 0}, {'id': 16897, 'quality': 4, 'icon': 'inv_chest_chain_16.jpg', 'itemLevel': 76}, {'id': 16903, 'quality': 4, 'icon': 'inv_belt_06.jpg', 'itemLevel': 76}, {'id': 16901, 'quality': 4, 'icon': 'inv_pants_06.jpg', 'itemLevel': 76}, {'id': 19892, 'quality': 3, 'icon': 'inv_boots_chain_13.jpg', 'itemLevel': 71}, {'id': 16904, 'quality': 4, 'icon': 'inv_bracer_03.jpg', 'itemLevel': 76}, {'id': 20661, 'quality': 2, 'icon': 'inv_gauntlets_05.jpg', 'itemLevel': 60, 'permanentEnchant': 1843}, {'id': 17110, 'quality': 4, 'icon': 'inv_jewelry_ring_21.jpg', 'itemLevel': 70}, {'id': 19863, 'quality': 3, 'icon': 'inv_jewelry_ring_47.jpg', 'itemLevel': 71}, {'id': 13965, 'quality': 3, 'icon': 'inv_misc_armorkit_09.jpg', 'itemLevel': 63}, {'id': 18470, 'quality': 3, 'icon': 'inv_jewelry_talisman_10.jpg', 'itemLevel': 62}, {'id': 18510, 'quality': 4, 'icon': 'inv_misc_cape_01.jpg', 'itemLevel': 62}, {'id': 22713, 'quality': 3, 'icon': 'inv_waepon_bow_zulgrub_d_01.jpg', 'itemLevel': 68}, {'id': 20694, 'quality': 2, 'icon': 'inv_misc_orb_04.jpg', 'itemLevel': 60}, {'id': 13289, 'quality': 1, 'icon': 'inv_weapon_rifle_05.jpg', 'itemLevel': 1}, {'id': 0, 'quality': 1, 'icon': 'inv_axe_02.jpg', 'itemLevel': 0}]",0.0,0.0,0.0,[],326.0,0.0,0.0,0.0,"[{'id': 0, 'icon': 'inv_axe_02.jpg'}, {'id': 0, 'icon': 'inv_axe_02.jpg'}, {'id': 0, 'icon': 'inv_axe_02.jpg'}, {'id': 0, 'icon': 'inv_axe_02.jpg'}]",1.0,0.0,0.0,402.0,81.0,[],0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.0,combatantinfo,293.0,2890.0,[],[],0.0,0.0,0.0,0.0,0.0,0.0,83.0,"[{'id': 22718, 'quality': 3, 'icon': 'inv_helmet_41.jpg', 'itemLevel': 68}, {'id': 15411, 'quality': 3, 'icon': 'inv_jewelry_talisman_07.jpg', 'itemLevel': 63}, {'id': 12082, 'quality': 2, 'icon': 'inv_shoulder_16.jpg', 'itemLevel': 58}, {'id': 127, 'quality': 2, 'icon': 'inv_shirt_07.jpg', 'itemLevel': 1}, {'id': 18530, 'quality': 3, 'icon': 'inv_chest_chain_12.jpg', 'itemLevel': 62, 'permanentEnchant': 928}, {'id': 16851, 'quality': 4, 'icon': 'inv_belt_28.jpg', 'itemLevel': 66}, {'id': 11749, 'quality': 3, 'icon': 'inv_pants_03.jpg', 'itemLevel': 53}, {'id': 16675, 'quality': 3, 'icon': 'inv_boots_plate_07.jpg', 'itemLevel': 59, 'permanentEnchant': 904}, {'id': 16850, 'quality': 4, 'icon': 'inv_bracer_17.jpg', 'itemLevel': 66, 'permanentEnchant': 247}, {'id': 13963, 'quality': 3, 'icon': 'inv_gauntlets_15.jpg', 'itemLevel': 60, 'permanentEnchant': 1887}, {'id': 22722, 'quality': 4, 'icon': 'inv_jewelry_ring_20.jpg', 'itemLevel': 65}, {'id': 9

,ability,amount,armor,attackPower,classResources,facing,hitPoints,hitType,itemLevel,mapID,maxHitPoints,mitigated,resourceActor,sourceID,sourceIsFriendly,spellPower,targetID,targetIsFriendly,timestamp,type,unmitigatedAmount,x,y
0,"{'name': 'Shoot Bow', 'guid': 2480, 'type': 1, 'abilityIcon': 'ability_marksmanship.jpg'}",116.0,0.0,0.0,"[{'amount': 0, 'max': 0, 'type': -1}]",228.0,100.0,1.0,63.0,0.0,100.0,62.0,2.0,21.0,1.0,0.0,27.0,0.0,0.0,damage,178.0,138008.0,-1229188.0


""


""


""


...

""


""


Connection Error: Event =  resources


""


""
